In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import time
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Bidirectional, LSTM, SimpleRNN,Input
from tensorflow.keras.optimizers import Adam


In [ ]:
df = pd.read_csv('../../dataset/final_dataset.csv')  

X = df[['Number of Lanes', 'Status of driver', 'Nature of environment', 'Velocity of vehicle', 'Distance between vehicles', 'Breaking capability']]
y = df['Collussion']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_scaled = np.reshape(X_scaled, (X_scaled.shape[0], 1, X_scaled.shape[1]))

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)


In [44]:
print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

(320000, 1, 6) (320000,) (80000, 1, 6) (80000,)


# Model

In [ ]:
def build_deep_rnn_bi_lstm():
    model = Sequential()

    model.add(Input(shape=(X_train.shape[1], X_train.shape[2])))

    model.add(SimpleRNN(units=100, activation='relu', return_sequences=True))
    
    model.add(Bidirectional(LSTM(units=100, activation='relu', return_sequences=True)))
    
    model.add(Bidirectional(LSTM(units=100)))

    model.add(Dense(units=1, activation='sigmoid'))

    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['mse', 'mae'])


    return model


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

best_weights_filepath ='..//models//rnn_bilstm.keras'

model_checkpoint = ModelCheckpoint(
    best_weights_filepath,  
    monitor='val_mse',  
    mode='min',       
    save_best_only=True,   
    verbose=1
)
model = build_deep_rnn_bi_lstm()
print(model.summary())




Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ simple_rnn_6 (SimpleRNN)        │ (None, 1, 100)         │        10,700 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_12                │ (None, 1, 200)         │       160,800 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_13                │ (None, 200)            │       240,800 │
│ (Bidirectional)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 1)              │           201 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 412,501 (1.57 MB)

 Trainable params: 412,501 (1.57 MB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
import time
start_time=time.time()
history = model.fit(
    X_train, y_train,
    epochs=50,
    batch_size=128,
    verbose=1,
    callbacks=[model_checkpoint],
    validation_data=(X_test, y_test)
)
end_time = time.time()

In [ ]:
total_training_time = end_time - start_time 
epochs = 50 
steps_per_epoch = 2000

total_steps = epochs * steps_per_epoch

time_per_step = total_training_time / total_steps  
time_per_step_ms = time_per_step * 1000  
time_per_step_us = time_per_step_ms * 1000

time_per_epoch = total_training_time / epochs

# Display results
print(f"Total Steps: {total_steps}")
print(f"Time per Step: {time_per_step:.6f} sec ({time_per_step_ms:.2f} ms)")
print(f"Time per Step: {time_per_step_us:.2f} µs")
print(f"Time per Epoch: {time_per_epoch:.2f} sec")


Total Steps: 100000
Time per Step: 0.019655 sec (19.66 ms)
Time per Step: 19655.27 µs
Time per Epoch: 39.31 sec


In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error for y_pred (MSE): {mse}')

rmse = np.sqrt(mse)
print(f'Root Mean Squared Error (RMSE): {rmse}')


y_pred = (y_pred > 0.5).astype(int)
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.16f}')


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(15, 5))

# Loss plot
plt.subplot(1, 3, 1)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper right')

# MSE plot
plt.subplot(1, 3, 2)
plt.plot(history.history['mse'], label='Train MSE')
plt.plot(history.history['val_mse'], label='Validation MSE')
plt.title('Mean Squared Error')
plt.ylabel('MSE')
plt.xlabel('Epoch')
plt.legend(loc='upper right')

# MAE plot
plt.subplot(1, 3, 3)
plt.plot(history.history['mae'], label='Train MAE')
plt.plot(history.history['val_mae'], label='Validation MAE')
plt.title('Mean Absolute Error')
plt.ylabel('MAE')
plt.xlabel('Epoch')
plt.legend(loc='upper right')

plt.tight_layout()
plt.show()
